In [ ]:
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
import fsl_mrs.utils.mrs_io as mrs_io
from fsl_mrs.utils.preproc import nifti_mrs_proc as proc


In [ ]:
from pathlib import Path

# Path to the directory 
NOTEBOOK_DIR = Path().resolve()
print(NOTEBOOK_DIR)
part = '004'

#Load in raw data following spec2nii (https://github.com/wtclarke/spec2nii) of the twix files
sup_data  = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_correction/raw_sLaser/{part}/sup_raw.nii.gz')
unsup_data = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_correction/raw_sLaser/{part}/unsup_raw.nii.gz')
#Data after system heating
#unsup_data = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_correction/raw_sLASER/{part}/unsup_raw_diffusion.nii.gz')


#Note: the water unsuppressed data is used as reference for coil combintation and eddy current correction
#By default it is identical to the normal water unsupprssed reference, just with more repeats
avg_ref_data = proc.average(unsup_data, 'DIM_DYN')


In [ ]:
#Preprocess the water suppressed data

N=4
N1=4

supp_data = proc.coilcombine(sup_data, reference=avg_ref_data)
eccc_data = proc.coilcombine(avg_ref_data, reference=avg_ref_data)

supp_data1=proc.apply_fixed_phase(supp_data, 180)
eccc_data1=proc.apply_fixed_phase(eccc_data, 180)

supp_data2 = proc.align(supp_data1, 'DIM_DYN', ppmlim=(0,4.2))
supp_data3 = proc.average(supp_data2, 'DIM_DYN')
supp_data5 = proc.truncate_or_pad(supp_data3, -N1, 'first')

limits = [-0.15,0.15]
limunits = 'ppm'

supp_data6 = proc.remove_peaks(supp_data5, limits, limit_units=limunits, figure=False)
supp_data7 = proc.shift_to_reference(supp_data6, 3.027, (2.9, 3.1))


final_sup = proc.phase_correct(supp_data7, (1.9, 2.1))
dataWS = np.conj(final_sup[0,0,0,:])


In [ ]:
#Plot suppressed data for sanity check 
plt.figure()
_ = final_sup.plot()

In [ ]:
#Preprocess water unsuppressed data

unsupp_data = proc.coilcombine(unsup_data, reference=avg_ref_data)
unsupp_data1=proc.apply_fixed_phase(unsupp_data, 180)
unsupp_data2 = proc.align(unsupp_data1, 'DIM_DYN', ppmlim=(0,9), figure=False )
unsupp_data3 = proc.average(unsupp_data2, 'DIM_DYN')
unsupp_data4 = proc.truncate_or_pad(unsupp_data3, -N, 'first')
unsupp_data4 = proc.phase_correct(unsupp_data4, (4.6,5))


plt.figure()
_=unsupp_data4.plot(ppmlim=(0,5))


In [ ]:
final_sup.save(rf'{NOTEBOOK_DIR}/Data_for_correction/sLaser_preproc/{part}/sup_preproc.nii.gz')
unsupp_data4.save(rf'{NOTEBOOK_DIR}/Data_for_correction/sLaser_preproc/{part}/unsup_preproc.nii.gz')
#unsupp_data4.save(rf'{NOTEBOOK_DIR}/Data_for_correction/sLaser_preproc/{part}/unsup_preproc_diffusion.nii.gz')